In [105]:
import gist

from PIL import Image
import imageio
from IPython.display import display, clear_output

import sys
sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
load_dotenv("../.env")
import csv

import numpy as np
import db_connection as dbcon

#%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set Parameters

In [106]:
# parameters
zoom = 15
lat = 30.4131736
lon = -87.600026
pixels = 640
samples_per_location = 1
precision = 4
num_images = 4
GMAPS_API_KEY = os.getenv('GMAPS_API_KEY')
IMG_FOLDER = os.getenv('MFP_IMG_FOLDER')

# connect to db
db = db_connection.connect("../credentials/mlab_db.txt","mfp")
images_lib_col = db["images_lib"]

# image dimensions
meters_per_px = imd.zoom_in_meters_per_pixel(zoom, lat)
image_size = meters_per_px*pixels ## in meters

print("Meters per pixel:", round(meters_per_px,2))
print("Image size:", round(image_size,2), "in meters")

DB Credentials from file
DB connected successfully!!!
Meters per pixel: 4.12
Image size: 2636.76 in meters


### Get filenames of all the files in Google Drive that correspond to Madrid

In [112]:
fnames = ima.get_filenames_of_city('US_1', IMG_FOLDER, 14)
fnames

['US_1_51.592016595967344_-116.1454855993385_14_600.png',
 'US_1_53.140069316272964_-103.49669505326648_14_600.png',
 'US_1_44.93618879833637_-115.09381259810687_14_600.png']

### Get image info of all the files of area name provided above (filenames, images, np.array)

In [116]:
images_info_list = ima.load_images_from_gdrive(fnames, IMG_FOLDER)

In [117]:
images_info_list = ima.add_labels_to_image_info(images_info_list)

In [126]:
try:
    _ = [print("Image", 
               images_info_list[i]['fname'], 
               "was provided with multi class label",
               images_info_list[i]['label_multi_pw']) for i in range(len(images_info_list))]
except:
    _ = [print("Image",
               images_info_list[i]['fname'],
               "was provided with multi class label",
               images_info_list[i]['label_multi_er']) for i in range(len(images_info_list))]

Image US_1_51.592016595967344_-116.1454855993385_14_600.png was provided with multi class label 0
Image US_1_53.140069316272964_-103.49669505326648_14_600.png was provided with multi class label 2
Image US_1_44.93618879833637_-115.09381259810687_14_600.png was provided with multi class label 0


In [127]:
try:
    _ = [print("Image", 
               images_info_list[i]['fname'], 
               "was provided with binary label",
               images_info_list[i]['label_binary_pw']) for i in range(len(images_info_list))]
except:
    _ = [print("Image",
               images_info_list[i]['fname'],
               "was provided with binary label",
               images_info_list[i]['label_binary_er']) for i in range(len(images_info_list))]

Image US_1_51.592016595967344_-116.1454855993385_14_600.png was provided with binary label 0
Image US_1_53.140069316272964_-103.49669505326648_14_600.png was provided with binary label 1
Image US_1_44.93618879833637_-115.09381259810687_14_600.png was provided with binary label 0


### Label images in Database

In [130]:
filename_upload_to_db = 'temp.csv'

save_labels_as_csv(images_info_list, IMG_FOLDER, 
                   output_name = filename_upload_to_db, 
                   label_multi_name = "label_multi_pw",
                   label_binary_name = "label_binary_pw")

In [131]:
write_labels_from_csv_to_db(images_lib_col, IMG_FOLDER, filename_upload_to_db)

4 labels added to db!


### Check labeling

In [165]:
label_name = 'label_binary_pw'
query = {label_name: '1'}

doc = images_lib_col.find(query)
for x in doc:
    keys = x.keys()
    for key in ['filename', label_name]:
        print(x[key])